In [ ]:
!pip install PyPDF2
!pip install pdfreader
!pip install pdfplumber

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.6/143.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 90.6 MB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: Pillow 8.4.0
    Uninstalling Pillow-8.4.0:
      Successfully uninstalled Pillow-8.4.0


In [ ]:
import PyPDF2
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import pdfplumber
import os

In [ ]:
# Путь к папке с PDF-файлами
pdf_folder_path = "/content/Docs/"

In [ ]:
# Проход по каждому PDF-файлу в папке и извлечение текста
for filename in os.listdir(pdf_folder_path):
    if filename.endswith(".pdf"):
        with pdfplumber.open(os.path.join(pdf_folder_path, filename)) as pdf:
            text = ""
            for page in pdf.pages:
                text += page.extract_text()
        # Удаление ненужных символов
        text = re.sub('[^а-яА-Я0-9\s]', '', text)
        # Токенизация текста
        tokens = word_tokenize(text)
        # Удаление стоп-слов
        stop_words = set(stopwords.words('russian'))
        tokens = [word for word in tokens if not word in stop_words]
        # Предобработка текста здесь

        # Сохранение текста в файл
        with open(os.path.join(pdf_folder_path, filename[:-4] + ".txt"), "w") as text_file:
            text_file.write(text)

In [ ]:
import tensorflow as tf
import numpy as np
import string

In [ ]:
# Загрузка документов для обучения модели
doc1 = open('/content/Docs/BMP-13.txt', 'r').read()
doc2 = open('/content/Docs/GOLD_Report_Russian_2014.txt', 'r').read()
doc3 = open('/content/Docs/federal_klinicheskie_rekomendaciy_hobl.txt', 'r').read()

In [ ]:
# Сбор всех документов в один
corpus = doc1 + doc2 + doc3

# Очистка текста от знаков пунктуации и приведение к нижнему регистру
corpus = corpus.lower().translate(str.maketrans('', '', string.punctuation))

# Разбиение на предложения
sentences = corpus.split('\n')

In [ ]:
# Создание токенизатора на основе текста
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(sentences)

In [ ]:
# Создание матрицы векторов предложений на основе токенизатора
input_sequences = []
for sentence in sentences:
    token_list = tokenizer.texts_to_sequences([sentence])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Максимальная длина последовательности входных данных
max_sequence_len = max([len(x) for x in input_sequences])

# Заполнение входных данных нулями, чтобы все последовательности были одинаковой длины
input_sequences = np.array(tf.keras.preprocessing.sequence.pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [ ]:
# Максимальная длина последовательности входных данных
max_sequence_len = max([len(x) for x in input_sequences])

# Заполнение входных данных нулями, чтобы все последовательности были одинаковой длины
input_sequences = np.array(tf.keras.preprocessing.sequence.pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Создание массива входных данных и меток
x_train = input_sequences[:, :-1]
y_train = input_sequences[:, -1]

# Создание модели Transformer
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=64, input_length=max_sequence_len-1))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Conv1D(64, 5, activation='relu'))
model.add(tf.keras.layers.MaxPooling1D(pool_size=4))
model.add(tf.keras.layers.LSTM(64))
model.add(tf.keras.layers.Dense(len(tokenizer.word_index)+1, activation='softmax'))

# Компиляция модели
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

# Обучение модели
model.fit(x_train, y_train, epochs=50, verbose=1)

Epoch 1/50
2925/2925 [==============================] - 268s 90ms/step - loss: 8.4957
Epoch 2/50
2925/2925 [==============================] - 256s 87ms/step - loss: 8.0315
Epoch 3/50
2925/2925 [==============================] - 248s 85ms/step - loss: 7.8253
Epoch 4/50
2925/2925 [==============================] - 250s 85ms/step - loss: 7.6141
Epoch 5/50
2925/2925 [==============================] - 248s 85ms/step - loss: 7.3677
Epoch 6/50
2925/2925 [==============================] - 251s 86ms/step - loss: 7.1062
Epoch 7/50
2925/2925 [==============================] - 250s 85ms/step - loss: 6.8525
Epoch 8/50
2925/2925 [==============================] - 250s 85ms/step - loss: 6.6133
Epoch 9/50
2925/2925 [==============================] - 248s 85ms/step - loss: 6.3783
Epoch 10/50
2925/2925 [==============================] - 249s 85ms/step - loss: 6.1641
Epoch 11/50
2925/2925 [==============================] - 248s 85ms/step - loss: 5.9659
Epoch 12/50
2925/2925 [=============================

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Загрузка предварительно обученной модели и токенизатора
#tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
#model = GPT2LMHeadModel.from_pretrained('gpt2')

# Описание функции для генерации текста на основе введенного текста
def generate_text(model, tokenizer, prompt, length):
    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    output = model.generate(input_ids=input_ids, max_length=length, do_sample=True)
    return tokenizer.decode(output[0], skip_special_tokens=True)


In [ ]:
# Использование функции для генерации текста
prompt = 'температура'
length = 500
recommendation = generate_text(model, tokenizer, prompt, length)
print(recommendation)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


температурату Эдытра пладовео защеступии: утлия просмер в дазаботчееск задь инік. Наеде бребы роспуй вбрениета. Увог разодя вачестя, за Соомека сэл, хорогима клэщя винтил влодой. Июлиеской просмер и видоноду ритемыниюческиа.тельных и всерудье нами ораврость оэвенкова. Сётурёщицию отьа бы прире и фирогецитно руший: помльное не доку ма маароенская руший. Наеде инік, клектуные Вушныв разодя всерудье правевительное сормытельное. Онаделя с навеже мушнаад, за Ру
